In [ ]:
from numpy import linspace, fromiter
from numpy import zeros, arange, ndarray
from matplotlib.pyplot import figure, rc
from typing import Callable
from math import sin, pi, e

In [ ]:
class CurveAdjust:
  def __init__(self, func: Callable, inter: tuple[int, int], m: int, n: int):
    (self.n1, self.n2), self.m, self.n = inter, m, n
    self.D = D = linspace(self.n1, self.n2, m)
    self.I = I = fromiter((func(x) for x in D), float, m)
    self.G = G = D.reshape(-1, 1) ** arange(n)
    self.A, self.B = (G.T @ G), (G.T @ I)
    self.fatoraçãoLU()

  def fatoraçãoLU(self):
    L = zeros((self.n, self.n))
    L.flat[::self.n + 1] = 1
    U = self.A

    for i1, i2 in arange(self.n-1).reshape(-1, 1) + (0, 1):
      L[i2:, i1] = U[i2:, i1] / U[i1, i1]
      U[i2:] -= L[i2:, i1, None] * U[i1]

    x, y = zeros(self.n), zeros(self.n)

    for i in arange(self.n):
      y[i] = self.B[i] - sum(L[i] * y)
    for i in arange(self.n)[::-1]:
      x[i] = (y[i] - sum(U[i] * x)) / U[i, i]

    self.RD = D = linspace(self.n1, self.n2, self.m)
    self.RI = (D.reshape(-1, 1) ** arange(self.n)) @ x

In [ ]:
class Integral:
  def __init__(self, func: Callable, inter: tuple[int, int], n: int):
    (i1, i2), self.func = inter, func
    self.D1, self.I1 = self.tuplas(i1, i2, 1000)
    adjust = CurveAdjust(func, (i1, i2), n, 10)
    (D2, I2), (D3, I3) = self.tuplas(i1, i2, n), (adjust.RD, adjust.RI)
    self.F1 = (D2, I2, self.trapezio(D2, I2), self.simpson(D2, I2))
    self.F2 = (D3, I3, self.trapezio(D3, I3), self.simpson(D3, I3))
    self.plotgfx()

  def tuplas(self, a: int, b: int, n: int):
    func, D = self.func, linspace(a, b, n)
    return D, fromiter((func(x) for x in D), float, n)

  def trapezio(self, D: ndarray, I: ndarray):
    diff = (D[1:] - D[:-1]).mean() / 2
    return diff * (2 * I.sum() - I[0] - I[-1])

  def simpson(self, D: ndarray, I: ndarray):
    diff = (D[1:] - D[:-1]).mean() / 3
    prt1, prt2 = I[1:-1:2].sum(), I[2:-2:2].sum()
    return diff * (I[0] + I[-1] + 4 * prt1 + 2 * prt2)

  def plotgfx(self):
    rc("font", family = "Consolas", size = 7.5)
    rc("lines", aa = 1, lw = .5)

    fig = figure(figsize = (12, 3))
    axes = fig.subplots(1, 2)
    fig.dpi = 400

    labels = ("Aproximado", "Ajustado")
    for (i, (D, I, R1, R2)) in enumerate((self.F1, self.F2)):
      axes[i].plot(self.D1, self.I1, label = "Original", c = "r")
      axes[i].plot(D, I, label = labels[i], c = "k")
      axes[i].fill_between(D, I, color = "k", alpha = .3)
      axes[i].set_title(f"Trapézio: {R1}, Simpson: {R2}")
      axes[i].set_xlabel("Domínio da função")
      axes[i].set_ylabel("Imagem da função")
      axes[i].legend()

    fig.suptitle("Integralização da Função")
    fig.tight_layout(pad = 1)

In [ ]:
Integral(lambda x: e**x, (1, 2), 5)
Integral(lambda x: e**x, (1, 2), 7)
Integral(lambda x: x**(1/2), (1, 4), 5)
Integral(lambda x: x**(1/2), (1, 4), 7)
Integral(lambda x: 1/(x**(1/2)), (2, 14), 5)
Integral(lambda x: 1/(x**(1/2)), (2, 14), 7)